In [1]:
%matplotlib inline
import matplotlib.pyplot as plt
import numpy as np
import os
import os.path as op
import shutil
from zipfile import ZipFile
from keras.preprocessing.image import array_to_img, img_to_array, load_img
from utils import get_image_paths, word_from_image_path, preprocess_image, print_im, TextTransform, N_CHARS, SEQUENCE_LENGTH, IMAGE_DIMENSIONS

%load_ext autoreload
%autoreload 2

Using TensorFlow backend.


In [2]:
from multi_gpu import make_parallel

In [3]:
images_paths = get_image_paths()

In [4]:
images = get_image_paths()

In [5]:
text_transformer = TextTransform()

### creating a batch

In [6]:
batch_y = text_transformer.make_batch_labels(images)

In [7]:
ims = [load_img(im, grayscale=True) for im in images]
ims = [preprocess_image(img_to_array(im)) for im in ims]
batch = np.array(ims)

In [8]:
batch.shape

(501, 32, 100)

In [9]:
batch_y.shape

(501, 23, 37)

### Model creation

In [10]:
from keras.models import Sequential
from keras.layers import Conv2D, MaxPooling2D, Dropout, Dense, Reshape, Activation
from keras.layers import Flatten

In [11]:
convolutions = [64, 128, 256, 512, 512]
kernels = [5, 5, 3, 3, 3]

model = Sequential()
input_shape = (None,) + IMAGE_DIMENSIONS + (1,)

model.add(Conv2D(nb_filter=64,
                     nb_row=kernels[0],
                     nb_col=kernels[0],
                     activation='relu',
                     border_mode='same',
                     batch_input_shape=input_shape, name="convo" + str(0)))

model.add(MaxPooling2D(pool_size=(2, 2), border_mode='same'))

for i, (kernel, convolution_size) in enumerate(zip(convolutions[1:], kernels[1:])):
    model.add(Conv2D(nb_filter=convolution_size,
                     nb_row=kernel,
                     nb_col=kernel,
                     activation='relu',
                     border_mode='same',
                     name="convo" + str(i + 1)))
    
    if i <= 3 :
        model.add(MaxPooling2D(pool_size=(2, 2), border_mode='same',))

model.add(Flatten())
# model.add(Dense(128, activation='relu'))
model.add(Dense(4096, activation='relu'))
model.add(Dense(4096, activation='relu'))

model.add(Dense(SEQUENCE_LENGTH * N_CHARS))

model.add(Reshape((SEQUENCE_LENGTH, N_CHARS)))
model.add(Activation('softmax'))


model.compile(optimizer='adam', loss='categorical_crossentropy')

In [12]:
p_model = make_parallel(model, 4)

In [17]:
p_model.compile(optimizer='adam', loss='categorical_crossentropy')

In [18]:
p_model.summary()

____________________________________________________________________________________________________
Layer (type)                     Output Shape          Param #     Connected to                     
convolution2d_input_1 (InputLaye (None, 32, 100, 1)    0                                            
____________________________________________________________________________________________________
lambda_1 (Lambda)                (None, 32, 100, 1)    0           convolution2d_input_1[0][0]      
____________________________________________________________________________________________________
lambda_2 (Lambda)                (None, 32, 100, 1)    0           convolution2d_input_1[0][0]      
____________________________________________________________________________________________________
lambda_3 (Lambda)                (None, 32, 100, 1)    0           convolution2d_input_1[0][0]      
___________________________________________________________________________________________

In [13]:
batch.shape

(501, 32, 100)

In [81]:
%%time
res = p_model.predict(batch[:32, :, :].reshape((batch[:32, :, :].shape[0],) + IMAGE_DIMENSIONS+ (1,)))

CPU times: user 84 ms, sys: 44 ms, total: 128 ms
Wall time: 457 ms


In [83]:
for i, by in enumerate(batch_y):
    try:
        print(text_transformer.word_from_matrix(res[i]), text_transformer.word_from_matrix(by))
    except:
        pass

feedings                feedings               
shortcake               shortcake              
gluiest                 gluiest                
immoderately            immoderately           
overspecialization      overspecialization     
dinah                   dinah                  
filbert                 filbert                
dogies                  dogies                 
bloe                    blog                   
dollies                 dollies                
reinvented              reinvented             
inflows                 inflows                
stridency               stridency              
shebang                 shebang                
separateness            separateness           
martyr                  martyr                 
refuter                 refuter                
perforating             perforating            
upstroke                upstroke               
unyoke                  unyoke                 
earners                 earners         

In [78]:
b = batch[:32]
b_y = batch_y[:32]

In [79]:
p_model.fit(b.reshape((b.shape[0],) + IMAGE_DIMENSIONS + (1,)), b_y, nb_epoch=300)

Epoch 1/300
32/32 [==============================] - 1s - loss: 0.4703
Epoch 2/300
32/32 [==============================] - 0s - loss: 0.4650
Epoch 3/300
32/32 [==============================] - 0s - loss: 0.4713
Epoch 4/300
32/32 [==============================] - 0s - loss: 0.4647
Epoch 5/300
32/32 [==============================] - 0s - loss: 0.4517
Epoch 6/300
32/32 [==============================] - 0s - loss: 0.4499
Epoch 7/300
32/32 [==============================] - 0s - loss: 0.4519
Epoch 8/300
32/32 [==============================] - 0s - loss: 0.4536
Epoch 9/300
32/32 [==============================] - 0s - loss: 0.4475
Epoch 10/300
32/32 [==============================] - 0s - loss: 0.4375
Epoch 11/300
32/32 [==============================] - 0s - loss: 0.4278
Epoch 12/300
32/32 [==============================] - 0s - loss: 0.4255
Epoch 13/300
32/32 [==============================] - 0s - loss: 0.4274
Epoch 14/300
32/32 [==============================] - 0s - loss: 0.4374
E

In [ ]:
import pickle

In [ ]:
pickle.dump()